#### 사용자 정의 모델 클래스
- 부모 클래스 : nn.Module
- 필수 오버라이딩
    * _init_() : 모델 층 구성, 즉 설계
    * forward() : 순방향 학습 진행 코드 구현

In [1]:
## 모듈 로딩
import torch                                    # 텐서 관련 모듈
import torch.nn as nn                           # 인공신경망 관련 모듈
import torch.nn.functional as F                 # 인공신경망 관련 함수들 모듈 (손실함수, 활성화 함수 등등)
import torch.optim as optim                     # 최적화 관련 모듈 (가중치, 절편 빠르게 찾아주는 알고리즘)

from torchinfo import summary                   # 모델 구조 및 정보 관련 모듈
from torchmetrics.regression import *           # 회귀 성능 지표 관련 모듈
from torchmetrics.classification import *       # 분류 성능 지표 관련 모듈

In [19]:
# 랜덤 고정
torch.manual_seed(1)

# 텐서 저장 및 실행 위치 설정
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'

- [기본] 신경망 클래스 <hr>
    * 입력층 - 입력 피처수 고정
    * 출력층 - 출력 타겟수 고정
    * 은닉층 - 고정

In [3]:
# 모델 설계
# 데이터셋 : 피처 4개, 타겟 1개, 회귀
# 입력층 : 입력 -   4개 / 출력 -  20개 / AF ReLU
# 은닉층 : 입력 -  20개 / 출력 - 100개 / AF ReLU
# 출력층 : 입력 - 100개 / 출력 -   1개 / AF X, Sigmoid & Softmax (분류)

In [4]:
# 입력 피처수가 고정인 모델
class MyModel(nn.Module):
    # 인스턴스/객체 생성 시 자동 호출되는 메서드 (콜백 함수 Callback func : 시스템에서 호출되는 함수)
    def __init__(self):
        # 부모 클래스 생성
        super().__init__()
        # 자식 클래스의 인스턴스 속성 설정
        self.input_layer = nn.Linear(4,20)      # W   4  + b 1 =>  1P ,  5*20 =   100개 변수 
        self.hidden_layer = nn.Linear(20,100)   # W  20 + b 1 =>  21P , 21*100 =  2100개 변수
        self.output_layer = nn.Linear(100,1)    # W 100 + b 1 => 101P

    # 순방향/전방향 학습 진행 시 자동 호출되는 메서드 (콜백 함수 Callback func : 시스템에서 호출되는 함수)
    # 전달 인자 : 학습용 데이터셋
    def forward(self, x):
        print('calling forward()')
        y = self.input_layer(x)  # 1개 퍼셉트론 : y=x1w1 + x2w2 + x3w3 + x4w4 + b ==> 20개
        y = F.relu(y)                #  0 <= y  ----> 죽은 relu ==> leakyReLU
        
        y = self.hidden_layer(y) # 1개 퍼셉트론 : y = x1w1 + x2w2 + ... + x20w20 + b ==> 100개
        y = F.relu(y)

        return self.output_layer(y) # 1개 퍼셉트론 : y = x1w1 + x2w2 + ... + x100w100 + b ==> 1개

In [12]:
# 입력 피처수가 동적인 모델
class MyModel2(nn.Module):
    # 인스턴스/객체 생성 시 자동 호출되는 메서드 (콜백 함수 Callback func : 시스템에서 호출되는 함수)
    def __init__(self, in_features):
        # 부모 클래스 생성
        super().__init__()
        # 자식 클래스의 인스턴스 속성 설정
        self.input_layer = nn.Linear(in_features,20)      # W   4  + b 1 =>  1P ,  5*20 =   100개 변수 
        self.hidden_layer = nn.Linear(20,100)   # W  20 + b 1 =>  21P , 21*100 =  2100개 변수
        self.output_layer = nn.Linear(100,1)    # W 100 + b 1 => 101P

    # 순방향/전방향 학습 진행 시 자동 호출되는 메서드 (콜백 함수 Callback func : 시스템에서 호출되는 함수)
    # 전달 인자 : 학습용 데이터셋
    def forward(self, x):
        print('calling forward()')
        y = self.input_layer(x)  # 1개 퍼셉트론 : y=x1w1 + x2w2 + x3w3 + x4w4 + b ==> 20개
        y = F.relu(y)                #  0 <= y  ----> 죽은 relu ==> leakyReLU
        
        y = self.hidden_layer(y) # 1개 퍼셉트론 : y = x1w1 + x2w2 + ... + x20w20 + b ==> 100개
        y = F.relu(y)

        return self.output_layer(y) # 1개 퍼셉트론 : y = x1w1 + x2w2 + ... + x100w100 + b ==> 1개

In [11]:
# 입력 피처수, 은닉층 퍼셉트론 수가 동적인 모델
class MyModel3(nn.Module):
    # 인스턴스/객체 생성 시 자동 호출되는 메서드 (콜백 함수 Callback func : 시스템에서 호출되는 함수)
    def __init__(self, in_features, in_out, h_out):
        # 부모 클래스 생성
        super().__init__()
        # 자식 클래스의 인스턴스 속성 설정
        self.input_layer = nn.Linear(in_features,in_out)      # W   4  + b 1 =>  1P ,  5*20 =   100개 변수 
        self.hidden_layer = nn.Linear(in_out,h_out)   # W  20 + b 1 =>  21P , 21*100 =  2100개 변수
        self.output_layer = nn.Linear(h_out,1)    # W 100 + b 1 => 101P

    # 순방향/전방향 학습 진행 시 자동 호출되는 메서드 (콜백 함수 Callback func : 시스템에서 호출되는 함수)
    # 전달 인자 : 학습용 데이터셋
    def forward(self, x):
        print('calling forward()')
        y = self.input_layer(x)  # 1개 퍼셉트론 : y=x1w1 + x2w2 + x3w3 + x4w4 + b ==> 20개
        y = F.relu(y)                #  0 <= y  ----> 죽은 relu ==> leakyReLU
        
        y = self.hidden_layer(y) # 1개 퍼셉트론 : y = x1w1 + x2w2 + ... + x20w20 + b ==> 100개
        y = F.relu(y)

        return self.output_layer(y) # 1개 퍼셉트론 : y = x1w1 + x2w2 + ... + x100w100 + b ==> 1개

In [ ]:
# 은닉층의 개수가 동적인 모델
class MyModel4(nn.Module):
    # 인스턴스/객체 생성 시 자동 호출되는 메서드 (콜백 함수 Callback func : 시스템에서 호출되는 함수)
    def __init__(self, in_features, in_out, h_out):
        # 부모 클래스 생성
        super().__init__()
        # 자식 클래스의 인스턴스 속성 설정
        self.input_layer = nn.Linear(in_features,in_out)      # W   4  + b 1 =>  1P ,  5*20 =   100개 변수
        self.hidden_layer = nn.Linear(in_out,h_out)   # W  20 + b 1 =>  21P , 21*100 =  2100개 변수
        self.output_layer = nn.Linear(h_out,1)    # W 100 + b 1 => 101P

    # 순방향/전방향 학습 진행 시 자동 호출되는 메서드 (콜백 함수 Callback func : 시스템에서 호출되는 함수)
    # 전달 인자 : 학습용 데이터셋
    def forward(self, x):
        print('calling forward()')
        y = self.input_layer(x)  # 1개 퍼셉트론 : y=x1w1 + x2w2 + x3w3 + x4w4 + b ==> 20개
        y = F.relu(y)                #  0 <= y  ----> 죽은 relu ==> leakyReLU
        
        y = self.hidden_layer(y) # 1개 퍼셉트론 : y = x1w1 + x2w2 + ... + x20w20 + b ==> 100개
        y = F.relu(y)

        return self.output_layer(y) # 1개 퍼셉트론 : y = x1w1 + x2w2 + ... + x100w100 + b ==> 1개

In [13]:
## 모델 인스턴스 생성
m1 = MyModel()
m2 = MyModel3(3,50,30)

### MyModel1

In [14]:
m1.named_parameters()

<generator object Module.named_parameters at 0x000002104605E5F0>

In [15]:
# 모델 파라미터, 즉 W와 b 확인
for m in m1.parameters():print(m)

Parameter containing:
tensor([[-0.1025,  0.4477, -0.4962, -0.0496],
        [ 0.2259,  0.3448,  0.0065, -0.2702],
        [-0.3865, -0.3786,  0.4026, -0.4354],
        [-0.1978,  0.2621,  0.2734, -0.4374],
        [ 0.0991,  0.0604,  0.4589, -0.0177],
        [ 0.2123, -0.0432,  0.4463, -0.0510],
        [-0.2938,  0.0922, -0.2647,  0.1250],
        [ 0.3476, -0.3889, -0.3638,  0.0068],
        [-0.0292, -0.1556,  0.4185,  0.2242],
        [-0.1658,  0.0725, -0.0752, -0.3201],
        [ 0.2046, -0.3299,  0.0605,  0.3394],
        [-0.4592,  0.0219,  0.4617,  0.2258],
        [-0.1972,  0.0787, -0.4695,  0.0803],
        [ 0.4347,  0.4345, -0.0080,  0.1813],
        [-0.0830, -0.1403,  0.0096, -0.2913],
        [ 0.2013,  0.2451, -0.1355, -0.1138],
        [-0.3484,  0.0639,  0.1989,  0.4747],
        [-0.4711, -0.4215,  0.0141,  0.4621],
        [ 0.0322,  0.1388,  0.0710, -0.0598],
        [ 0.3569,  0.3670, -0.2883, -0.2994]], requires_grad=True)
Parameter containing:
tensor([-0.3888

In [10]:
# 학습 진행 ==> 모델 인스턴스명(데이터)
# 임의의 데이터
dataTS = torch.FloatTensor([[1,3,5,7],[2,4,6,8]])
targetTS = torch.FloatTensor([[4],[5]])

# 학습
pre_y = m1(dataTS)
print(pre_y)

calling forward()
tensor([[-0.1395],
        [-0.1858]], grad_fn=<AddmmBackward0>)


### MyModel3

In [16]:
# 모델 파라미터, 즉 W와 b 확인
for m in m2.parameters():print(m)

Parameter containing:
tensor([[-0.1055,  0.1731,  0.3126],
        [ 0.1047,  0.2473,  0.4548],
        [ 0.3073,  0.2808,  0.5694],
        [ 0.1517,  0.4245,  0.3458],
        [-0.4783, -0.3805,  0.4100],
        [ 0.5529, -0.2264,  0.2219],
        [ 0.4732, -0.4674, -0.2097],
        [ 0.5043, -0.0995,  0.1917],
        [ 0.1908,  0.1261,  0.5377],
        [ 0.5048, -0.4805,  0.1511],
        [ 0.1609,  0.0817, -0.1890],
        [ 0.2374, -0.1757,  0.2817],
        [ 0.1493,  0.3702, -0.0461],
        [-0.1539,  0.4455,  0.3673],
        [ 0.2049, -0.5536,  0.2376],
        [ 0.0480, -0.5712, -0.5656],
        [ 0.0847, -0.1088,  0.4414],
        [ 0.0405, -0.4604, -0.0941],
        [ 0.3139,  0.1358,  0.5173],
        [ 0.4296,  0.1824,  0.5300],
        [-0.3494,  0.0475, -0.1926],
        [-0.1424,  0.0779,  0.3946],
        [-0.1145,  0.2290,  0.4468],
        [ 0.0802, -0.0741, -0.4250],
        [-0.3210,  0.0176,  0.1149],
        [-0.0788, -0.5200, -0.3425],
        [ 0.3381

In [18]:
# 학습 진행 ==> 모델 인스턴스명(데이터)
# 임의의 데이터
dataTS = torch.FloatTensor([[1,3,5],[2,4,6]])
targetTS = torch.FloatTensor([[4],[5]])

# 학습
pre_y = m2(dataTS)
print(pre_y)

calling forward()
tensor([[-0.4719],
        [-0.5332]], grad_fn=<AddmmBackward0>)
